# Keras preprocessing layers

There is usually need to preprocess data for neural network.

So far we were using preprocessing done ahead using Python script and transforming data using NumPy or Pandas

We also did preprocessing on the fly on tf.data.Dataset using map function.

Another approach is including preprocessing layers directly inside the model.

This is benefical becasue you are preprocessing data on the fly and model preprocesses data the same way during training and production.

https://keras.io/api/layers/preprocessing_layers/

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import datasets, layers
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Load data for preprocessing demonstration.

In [ ]:
URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
COLUMN_NAMES = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']

dataset = pd.read_csv(
    URL, 
    names=COLUMN_NAMES,
    na_values='?', 
    sep=' ',
    comment='\t',  
    skipinitialspace=True)

dataset = dataset.dropna()

In [ ]:
train_dataset = dataset.sample(frac = 0.8, random_state = 42)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

### Normalization layer

Standardize inputs by calling adapt function.

It is possible to set mean and variance by hand as well.

In [ ]:
norm_layer = tf.keras.layers.Normalization()
# you don't need to pass whole dataset to adapt. It could be enought to put big enough randomly sampled data.
norm_layer.adapt(train_dataset)
norm_layer(train_dataset)

Setting normalization layers eliminates risk of data pre-processing mismatch.

It also computes the normalization using GPU instead of CPU, co it can run faster.

In [ ]:
input_layer = tf.keras.layers.Input(shape=(7,))
x = norm_layer(input_layer)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
output_layer = tf.keras.layers.Dense(1)(x)
model = Model(inputs = input_layer, outputs = output_layer)

In [ ]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mean_absolute_error'])

In [ ]:
%%time
history = model.fit(
    train_dataset,
    train_labels,
    validation_split=0.2,
    verbose=1, 
    epochs=100)

The preprocessing works with tf.data.Dataset as well

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_dataset, train_labels)).batch(5)
dataset = dataset.map(lambda X, y: (norm_layer(X), y))

In [ ]:
list(dataset.take(1))

## Image Preprocessing Layers

Preprocessing layers for image inputs.

### Rescaling layer

Rescales the pixel values.

* scale: scale to apply on inputs - 1/255 will scale [0, 255] to [0, 1]
* offset: the offset - if you would like to have new scale from [-1, 1] - you should use 2/255 with offset -1

https://keras.io/api/layers/preprocessing_layers/image_preprocessing/rescaling/

In [ ]:
# load the cifar10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [ ]:
x = layers.Rescaling(1.0 / 255)
x(train_images[0])

### Resizing layer

Resize images on input.

https://keras.io/api/layers/preprocessing_layers/image_preprocessing/resizing/

In [ ]:
x = layers.Resizing(224, 224)
x(train_images[0])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(x(train_images[0])/255)

## Image augmentation

Data augmentation is a regularization technique that artificially increases the size of the training set by generating multiple variants of single training instance.

Augmentation could help overfitting.

Augmented data should be as realistic looking as possible (human should not be able to tell which of the data has been made artifically).

Augmentation forces the model to learn general patterns by introducing variations in the position, orientation or size of the objects.

Listed transformation layers are active only during the training, randomly apply some transformation to batch of images.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
plt.show()

### RandomRotation

Randomly rotates images during training.

https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomRotation

In [ ]:
# factor is fraction of 2Pi - 360° degrees
# could be a scalar or tuple of size 2 representing lower and upper bound for rotating clockwise and counter-clockwise
x = tf.keras.layers.RandomRotation(factor=0.05, seed=42)
X_train_scaled = x(train_images[:25])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # rotated pictures do not use ints for color mapping, so you need to either crop the decimals or normalize it
    # to 0-1 for matplotlib to be able to display them
    plt.imshow(X_train_scaled[i]/255)
plt.show()

### RandomFlip

Randomly flip each image horizontally and/or vertically.

https://keras.io/api/layers/preprocessing_layers/image_preprocessing/random_flip/

In [ ]:
x = tf.keras.layers.RandomFlip(mode="horizontal", seed=42)
X_train_scaled = x(train_images[:25])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_scaled[i]/255)
plt.show()

### RandomContrast

Randomly adjusts contrast during training - can simulate worse light conditions.

https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_contrast/

In [ ]:
x = tf.keras.layers.RandomContrast(factor=0.8, seed=42)
X_train_scaled = x(train_images[:25])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_scaled[i]/255)
plt.show()

### RandomZoom

Randomly zoom each image during training.

https://keras.io/api/layers/preprocessing_layers/image_preprocessing/random_zoom/

In [ ]:
x = tf.keras.layers.RandomZoom(height_factor=(0.5), width_factor=(0.5), seed=42)
X_train_scaled = x(train_images[:1000])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_scaled[i]/255)
plt.show()

### Using augmentations in model

We can either use the the preprocessing in the model or using tf.data.Dataset

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=42),
    tf.keras.layers.RandomRotation(factor=0.05, seed=42),
    tf.keras.layers.RandomContrast(factor=0.2, seed=42)
])

In [ ]:
input_layer = layers.Input(shape=(32, 32, 3))
x = data_augmentation(input_layer)
x = layers.Rescaling(1.0 / 255)(x)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
output_layer = layers.Dense(10, activation='softmax')(x)
model = Model(inputs = input_layer, outputs = output_layer)
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, epochs=3, validation_data=(test_images, test_labels))

### Using augmentations on dataset

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.batch(32).map(lambda x, y: (data_augmentation(x), y))

In [ ]:
input_layer = layers.Input(shape=(32, 32, 3))
x = layers.Rescaling(1.0 / 255)(input_layer)
x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
output_layer = layers.Dense(10, activation='softmax')(x)
model = Model(inputs = input_layer, outputs = output_layer)
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(dataset, epochs=3, validation_data=(test_images, test_labels))